In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
import librosa
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import SpectralProcessing
from SpectralProcessing import *
import Conv1dGLU
from Conv1dGLU import Conv1dGLU
import Cloning_Samples_Attention
from Cloning_Samples_Attention import *
import Encoder
from Encoder import *

In [2]:
batch_size = 25
N_samples = 25
frequency_bands = 80
hop_length = 400
window_size = 1600

In [3]:
z = torch.randn(25,25,100,80)
z = Variable(z.type(torch.FloatTensor))

In [8]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.prenet = PreNet()
        self.conv = Conv1dGLU()
        self.prehead = PreEncoderLayer()
        self.attention = Attention(128, [128,128])
        self.bn = nn.BatchNorm1d(N_samples)
        
    def forward(self, x):
        batch_size, N_samples = x.size(0), x.size(1)
        x = self.prenet(x)
        x = x.view(batch_size*N_samples, x.size(2), x.size(3)).transpose(1,2)
        x = self.conv(x)
        x = x.transpose(1,2).view(batch_size,N_samples,x.size(1),x.size(2))
        x = librosa.decompose.hpss(x)[0]
        x = x.mean(axis=2)
        conv_out = x
        x = prehead(x)
        x = self.attention(x)
        x = self.bn(nn.Softsign(x))
        x *= conv_out
        return x

In [9]:
attn = Attention(128,[128,128])

In [10]:
enc = Encoder()